# Trail scrapping project

This project is designed to generate a table from scrapped data on a website. This assessment will be looking at data from HikingProject for the National Parks. This is designed to be used when a region gets upgraded to a National Park. By comparing to this database, it is hoped that it will help identify trails that will see a spike in users in the new National Park so proper maintentance and security can be allocated for this uptick. 

In [87]:
import os
from selenium import webdriver
import pandas as pd
import csv
import time
from bs4 import BeautifulSoup as bs
import numpy as np
import matplotlib as plt

In [54]:
#set up Selenium to access the web driver

In [162]:
chromedriver='/Applications/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver
driver=webdriver.Chrome(chromedriver)

/var/folders/z4/gqr7pbzd1jx381lnrbtb0c4w0000gp/T/ipykernel_963/2050704745.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(chromedriver)


In [59]:

driver.get('https://www.hikingproject.com/search?q=national%20park')
area_button=driver.find_element_by_xpath('//a[contains(.,"Area")]')
area_button.click()

#this brings Chrome to the list of national park pages on hikingproject's directory. Next step is click "Load More" button.

/var/folders/z4/gqr7pbzd1jx381lnrbtb0c4w0000gp/T/ipykernel_963/1792656644.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(chromedriver)
/var/folders/z4/gqr7pbzd1jx381lnrbtb0c4w0000gp/T/ipykernel_963/1792656644.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  area_button=driver.find_element_by_xpath('//a[contains(.,"Area")]')


In [60]:
load_more_button=driver.find_element_by_xpath('//a[contains(.,"Load")]')
load_more_button.click()
#this makes sure all the links for all the US parks on Hikerproject show up. 

/var/folders/z4/gqr7pbzd1jx381lnrbtb0c4w0000gp/T/ipykernel_963/101422780.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  load_more_button=driver.find_element_by_xpath('//a[contains(.,"Load")]')


There are no more links to click. The next step is to derive a formula to extra all the links in a list. First convert the website as is to Beautiful soup

In [61]:
PL_soup=bs(driver.page_source)

In [62]:
def get_site_list(header, soup):
    
    site_list=[link['href'] for head in soup.find_all(header) for link in head.find_all('a')]
    return site_list

In [63]:
park_site_list=get_site_list('strong', PL_soup)

In [64]:
#only the first 53 sites are needed as after that they are not US National Parks.
park_site_list=park_site_list[0:53]
park_site_list[-1]

'/directory/8009611/great-smoky-mountains-national-park?search=1&type=area&method=resultsPage&query=national%20park'

Looking at the links above, the links can be cleaned up. Everything after the questionmark can also be deleted. 

In [65]:
park_site_list=[site.split('?')[0] for site in park_site_list]
park_site_list=['https://www.hikingproject.com/'+site for site in park_site_list]
park_site_list[0:5]

['https://www.hikingproject.com//directory/8011077/yellowstone-national-park',
 'https://www.hikingproject.com//directory/8007144/redwood-national-park',
 'https://www.hikingproject.com//directory/8007222/sequoia-national-park',
 'https://www.hikingproject.com//directory/8007223/yosemite-national-park',
 'https://www.hikingproject.com//directory/8007303/pinnacles-national-park']

The next step is to go to each National Park's site, and find the website for each trail. First, similar to above, a button must be clicked several times to access more data. Unlike above, this task requires clicking on the link until the link disappears. This is because this assessment needs to scrape the websites for each trail at the national park so the program needs to keep clicking until no more trail links are available to load. First, save the above list of National Park links so they do not need to be scraped again.

In [66]:
#To avoid risk of losing data, save the list of parks to a csv. 
with open('/Users/scottturransky1/Documents/2022_metis/park_link_list.csv', 'w') as pll:
    writer=csv.writer(pll)
    writer.writerows(park_site_list)



In [1]:
#be sure to close the driver before running it again.
#driver.close()

# Second website list generation

Previously, the list of national parks links were saved to a CSV file. This step starts with opening the CSV file and getting the list (so if the Jupyter notebook was closed, it could just start here. 

In [68]:
nat_park_list2=[]
with open ('park_link_list.csv') as csv_file:
    csv_reader=csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        nat_park_list2.append("".join(row))
nat_park_list2

['https://www.hikingproject.com//directory/8011077/yellowstone-national-park',
 'https://www.hikingproject.com//directory/8007144/redwood-national-park',
 'https://www.hikingproject.com//directory/8007222/sequoia-national-park',
 'https://www.hikingproject.com//directory/8007223/yosemite-national-park',
 'https://www.hikingproject.com//directory/8007303/pinnacles-national-park',
 'https://www.hikingproject.com//directory/8007346/joshua-tree-national-park',
 'https://www.hikingproject.com//directory/8007347/death-valley-national-park',
 'https://www.hikingproject.com//directory/8007858/everglades-national-park',
 'https://www.hikingproject.com//directory/8008003/haleakala-national-park',
 'https://www.hikingproject.com//directory/8008816/voyageurs-national-park',
 'https://www.hikingproject.com//directory/8009003/glacier-national-park',
 'https://www.hikingproject.com//directory/8009873/crater-lake-national-park',
 'https://www.hikingproject.com//directory/8010544/zion-national-park',
 

The next step is find the different trails at each site. This code is written in case the process stopped midway through. This way if the has already found data for a park, it will not be run again (this is the if/else statement at the beginning). A dictionary will be created. The keys will be the national park website, and the value for each key (NP) will be a list of all the trail websites at that park. 

In [158]:
np_trail_dict={}

In [188]:
for park in nat_park_list2:
    npark=park.split('/')[-1]
    chromedriver='/Applications/chromedriver'
    os.environ['webdriver.chrome.driver'] = chromedriver
    driver=webdriver.Chrome(chromedriver)
    if npark in np_trail_dict.keys():
        continue
    else:
        driver.get(park)
        while True:
            time.sleep(1)
            try:
                get_more_button=driver.find_element_by_xpath('//button[@id="load-more-trails"]')
                get_more_button.click()
            except:
                break
        trail_soup=bs(driver.page_source)
        try:
            temp_trail_list=[link['href'] for link in trail_soup.find(class_='table table-striped trail-table').find_all('a')]
        except AttributeError:
            print(park)
        np_trail_dict[npark]=temp_trail_list



/var/folders/z4/gqr7pbzd1jx381lnrbtb0c4w0000gp/T/ipykernel_963/2162473958.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(chromedriver)
/var/folders/z4/gqr7pbzd1jx381lnrbtb0c4w0000gp/T/ipykernel_963/2162473958.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  get_more_button=driver.find_element_by_xpath('//button[@id="load-more-trails"]')


In [193]:
numtrails=0
for park in np_trail_dict.values():
    numtrails+=len(park)
numtrails

2939

In [189]:
driver.close()

In [200]:
#To avoid risk of losing data, save the list of trails to a csv. 
with open('/Users/scottturransky1/Documents/2022_metis/trail_websites.csv', 'w') as tw:
    writer=csv.writer(tw)
    for key, value in np_trail_dict.items():
        writer.writerow([key,value])


The next step is create the dataframe that will be used for the linear analysis. First, create columns for the trail name, national park name, and the trail website. That will be done in a new Notebook to avoid running the above code multiple times. It will start with the importing the CSV just written. 

In [201]:
%store np_trail_dict

Stored 'np_trail_dict' (dict)
